In [2]:
import os
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
 

# Get Data

Get DI Data from ``Spline.xlsx`` and resample it to Business Month End Frequency

In [3]:
#Get original data on DI futures from BMF

DataBaseTable = pd.read_excel('Splines.xlsx',index_col=0)
DataBaseTable = DataBaseTable.resample('BM').last()
DataBaseTable.head()

,1,11,12,13,14,15,16,17,18,19,...,51,52,53,54,55,56,57,58,59,60
2007-03-30,11.871196,11.836765,11.788089,11.752256,11.727396,11.708699,11.692097,11.675767,11.658307,11.638825,...,11.578391,11.576099,11.572998,11.569447,11.565804,11.562428,11.559677,11.557820,11.556824,11.556585
2007-04-30,11.543392,11.345224,11.279222,11.216923,11.160814,11.112620,11.071417,11.035708,11.004216,10.976842,...,10.596224,10.593190,10.590143,10.587102,10.584088,10.581123,10.578227,10.575423,10.572733,10.570177
2007-05-31,11.387237,10.983609,10.928755,10.877533,10.826560,10.775333,10.723884,10.672835,10.625515,10.586037,...,10.170549,10.168078,10.165988,10.163686,10.160579,10.156296,10.151245,10.146001,10.140992,10.136128
2007-06-29,10.932925,10.739882,10.710822,10.690869,10.674815,10.656605,10.631937,10.605875,10.586587,10.580970,...,10.699415,10.700368,10.699915,10.698095,10.694946,10.690507,10.684817,10.677914,10.669837,10.660624
2007-07-31,10.590793,10.888898,10.883627,10.883162,10.885837,10.890248,10.896204,10.903869,10.913484,10.925786,...,11.052343,11.049802,11.052050,11.059557,11.068765,11.075243,11.076036,11.073316,11.070362,11.069489


Get list of dates

In [5]:
calendar = sorted(list(set([d for d in DataBaseTable.index if d>=pd.to_datetime('30-Mar-2007')])))

Create empty DataFrames

In [6]:
ExcessReturns = pd.DataFrame(index=calendar[1:],columns=DataBaseTable.columns[2:])
ForwardBias = pd.DataFrame(index=calendar[1:],columns=DataBaseTable.columns[2:])
CPForwards = pd.DataFrame(index=calendar[1:],columns=['ER'] + [12,24,36,48,50])
 

In [ ]:
for d,d_minus_1 in zip(calendar[1:],calendar[:-1]):
    # print(d.strftime('%Y%m%d'))
    CP_ER = 0.
    for n in DataBaseTable.columns[2:]:
        P_n_minus_1_d = 100./((1.+DataBaseTable.loc[d,n-1]/100.)**((n-1)/12.))
        P_n_d_minus_1 = 100. / ((1. + DataBaseTable.loc[d_minus_1, n] / 100.) ** (n / 12.))
        P_n_minus_1_d_minus_1 = 100. / ((1. + DataBaseTable.loc[d_minus_1, n-1] / 100.) ** ((n-1) / 12.))
 
        ret_n_d = P_n_minus_1_d/P_n_d_minus_1-1.
        ret_risk_free = (1.+DataBaseTable.loc[d_minus_1,1]/100.)**(1/12.)-1.
        roll_yield = P_n_minus_1_d_minus_1/P_n_d_minus_1-1.
        ExcessReturns.loc[d,n] = ret_n_d - ret_risk_free
        ForwardBias.loc[d,n] = roll_yield - ret_risk_free
        if n in CPForwards.columns:
            CPForwards.loc[d,n] = roll_yield
            if n in [24,36,48,50]:
                CP_ER += ExcessReturns.loc[d,n]/4.
    CPForwards.loc[d, 'ER'] = CP_ER

In [ ]:


 


 
writer = pd.ExcelWriter(os.path.join(pathname,'Factors.xlsx'))
ExcessReturns.sort_index().to_excel(writer,'Returns')
ForwardBias.sort_index().to_excel(writer,'FamaBlissFactors')
 
#Fama Bliss Regression
for n in [12,24,36,48,60]:
    Y = ExcessReturns[n].dropna().astype(float).to_frame('ER')
    X = ForwardBias[n].dropna().astype(float).to_frame('FB')
    observations = [i for i in Y.index if i in X.index]
    df = pd.concat([Y.loc[observations],X.loc[observations]],axis=1)
    lm = smf.ols(formula='ER ~ FB', data=df).fit()
    print('Fama-Bliss Factors regression for n=%s months' % str(n))
    print(lm.summary())
    print('#######################################################')
 
#Cochrane and Piazzesi
CPForwards = CPForwards.dropna(how='any').astype(float)
CPForwards.columns = ['ER'] + ['F'+ str(x) for x in CPForwards.columns[1:]]
lm = smf.ols(formula='ER ~ F24 + F36 + F48 + F50', data=CPForwards).fit()
CP = lm.fittedvalues
CP.sort_index().to_frame('CP_Factor').to_excel(writer,'CP_single_factor')
 
for n in [12,24,36,48,60]:
    Y = ExcessReturns[n].dropna().astype(float).to_frame('ER')
    X = CP.dropna().astype(float).to_frame('CP')
    observations = [i for i in Y.index if i in X.index]
    df = pd.concat([Y.loc[observations],X.loc[observations]],axis=1)
    lm = smf.ols(formula='ER ~ CP', data=df).fit()
    print('Cochrane and Piazzesi Factor regression for n=%s months' % str(n))
    print(lm.summary())
    print('#######################################################')
 
#PCA factors
COV = DataBaseTable[DataBaseTable.columns[2:]].diff(1).cov().values
eig_val_cov, eig_vec_cov = np.linalg.eig(COV)
PCAs = pd.DataFrame(index=range(1,len(eig_val_cov)+1),data=-1*eig_vec_cov,columns = ['PC' + str(x) for x in range(1,len(eig_val_cov)+1)])
PCAFactors = pd.DataFrame(np.dot(np.matrix(DataBaseTable[DataBaseTable.columns[2:]].diff(1).dropna()),np.matrix(PCAs.iloc[:,:4])))
PCAFactors.index = DataBaseTable[DataBaseTable.columns[2:]].diff(1).dropna().index
PCAFactors.columns = ['PC' + str(x) for x in range(1,5)]
PCAFactors['ER'] = CPForwards['ER']
PCAFactors.sort_index().to_excel(writer,'PCA_factors')
lm = smf.ols(formula='ER ~ PC1 + PC2 + PC3 + PC4', data=PCAFactors).fit()
PCApredictor = lm.fittedvalues
for n in [12,24,36,48,60]:
    Y = ExcessReturns[n].dropna().astype(float).to_frame('ER')
    X = PCApredictor.dropna().astype(float).to_frame('LN')
    observations = [i for i in Y.index if i in X.index]
    df = pd.concat([Y.loc[observations],X.loc[observations]],axis=1)
    lm = smf.ols(formula='ER ~ LN', data=df).fit()
    print('PCA Factor regression for n=%s months' % str(n))
    print(lm.summary())
    print('#######################################################')
writer.save()
